# Simularium Conversion Tutorial : Smoldyn Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.smoldyn import SmoldynConverter, SmoldynData
from simulariumio import UnitData, MetaData, DisplayData, DISPLAY_TYPE, ModelMetaData, InputFileData
from simulariumio.filters import TranslateFilter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `SmoldynConverter` consumes spatiotemporal data from Smoldyn. 

The converter requires a `SmoldynData` object as a parameter ([see documentation](https://allen-cell-animated.github.io/simulariumio/simulariumio.smoldyn.html#simulariumio.smoldyn.smoldyn_data.SmoldynData)).

If you'd like to specify PDB or OBJ files or color for rendering an agent type, add a `DisplayData` object for that agent type, as shown below ([see documentation](https://allen-cell-animated.github.io/simulariumio/simulariumio.data_objects.html#module-simulariumio.data_objects.display_data)).


In [2]:
box_size = 100.

example_data = SmoldynData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/allen-cell-animated/simulariumio",
            source_code_license_url="https://github.com/allen-cell-animated/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    smoldyn_file=InputFileData(
        file_path="../simulariumio/tests/data/smoldyn/example_3D.txt",
    ),
    display_data={
        "red(solution)": DisplayData(
            name="B",
            radius=1.0,
            display_type=DISPLAY_TYPE.OBJ,
            url="b.obj",
            color="#dfdacd",
        ),
        "green(solution)": DisplayData(
            name="A",
            radius=2.0,
            color="#0080ff",
        ),
    },
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

(since this model's coordinates are all positive, use a `TranslateFilter` to center the data in the viewer.)

In [3]:
c = SmoldynConverter(example_data)
translation_magnitude = -box_size / 2
filtered_data = c.filter_data([
    TranslateFilter(
        translation_per_type={},
        default_translation=translation_magnitude * np.ones(3)
    ),
])
c.write_external_JSON(filtered_data, "example_smoldyn")

Reading Smoldyn Data -------------
Filtering: translation -------------
Writing JSON (external)-------------
Converting Trajectory Data -------------
saved to example_smoldyn.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.